# Introduction

This project will explore the OCCP data. Open Charge Point Protocol (OCPP) is an open standard communication protocol for Electric Vehicle (EV) charging stations. It defines interactions between EV charging stations and a central system, helping to facilitate security, transactions, diagnostics, and more.

This dataset if from OCCP v1.6

## Charging System Diagram
Organization < Property < Location < Cluster < Station < UserID

A cluster is a grouping of chargers/stations. This for convenience/load balancing

Each circuit can have multiple clusters.

Each cluster has its own breaker


## Prepare Enviornment

In [1]:
# Access to Google Drive
# This seems to propagate credentials better from its own cell

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Packages and methods

!pip install PyGithub
from github import Github
import os
import datetime
from google.colab import userdata

!pip install pandas pyxlsb
import pandas as pd

import numpy as np

import sys
import logging
import psycopg2

!pip install SQLAlchemy psycopg2-binary
import seaborn as sns
import json

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import matplotlib.pyplot as plt

from datetime import timedelta
import holidays

!pip install statsmodels
import statsmodels.api as sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 32.0 MB/s eta 0:00:00


In [ ]:
# Update github

def colab_to_github(notebook_path, github_repo, folder_path=None, commit_message=None, branch="main"):
   try:
       print("Fetching GitHub token...")
       token = os.getenv('GITHUB_TOKEN')
       if not token:
           raise ValueError("GitHub token is missing or invalid. Ensure it is set as an environment variable.")

       # Add debug logging (only showing first few chars for security)
       print(f"Token format check - starts with: {token[:4]}")

       print("Token successfully retrieved.")
       g = Github(token)
       repo = g.get_repo(github_repo)
       print(f"Connected to repository: {github_repo}")

       if not commit_message:
           commit_message = f"Auto-commit from Colab: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
       print(f"Using commit message: {commit_message}")

       with open(notebook_path, 'r') as file:
           notebook_content = file.read()
       print(f"Notebook content read from {notebook_path}")

       filename = os.path.basename(notebook_path)
       # Construct the full file path including the folder if specified
       file_path = f"{folder_path}/{filename}" if folder_path else filename
       print(f"Target file path in repo: {file_path}")

       try:
           print(f"Checking if file exists at {file_path}...")
           existing_file = repo.get_contents(file_path, ref=branch)
           repo.update_file(
               path=file_path,
               message=commit_message,
               content=notebook_content,
               sha=existing_file.sha,
               branch=branch
           )
           print(f"File updated successfully in branch '{branch}'.")
       except Exception:
           print(f"File does not exist at {file_path}. Attempting to create...")
           repo.create_file(
               path=file_path,
               message=commit_message,
               content=notebook_content,
               branch=branch
           )
           print(f"File created successfully in branch '{branch}'.")

   except Exception as e:
       print(f"Error occurred: {e}")

raw_token = userdata.get('GITHUB_TOKEN')
cleaned_token = raw_token.replace('token ', '').strip()
print(f"Cleaned token starts with: {cleaned_token[:4]}")

os.environ['GITHUB_TOKEN'] = cleaned_token

# Call the function
notebook_path = "/content/drive/MyDrive/Colab Notebooks/OCCP.ipynb"
github_repo = "davidelgas/DataSciencePortfolio"  # This is the correct repository path
folder_path = "OCCP"  # This specifies the directory within the repository
commit_message = "Updated notebook from Colab"

colab_to_github(notebook_path, github_repo, folder_path, commit_message)

## Ingest raw data

In [ ]:
# import log data

import pandas as pd
import numpy as np

def load_file(file_path):
    """Load a single CSV file."""
    return pd.read_csv(file_path)

def concatenate_files(file_paths):
    """Load and combine multiple CSV files."""
    dfs = []
    for file_path in file_paths:
        df = load_file(file_path)
        if not df.empty:
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

if __name__ == "__main__":
    file_paths = [
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/sept_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/oct_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/nov_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/dec_100_sample.csv'
    ]

    # Concatenate all files
    df_logs = concatenate_files(file_paths)

    # Save the combined raw data
    df_logs.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs.pkl')

In [ ]:
#Import property tables from AWS

import psycopg2
import pandas as pd
import os

# Load credentials
def load_credentials(path_to_credentials):
    with open(path_to_credentials, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

# Connection parameters
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Connect and fetch data
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Fetch properties table
cursor.execute("SELECT * FROM properties;")
df_prop = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df_prop.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.csv', index=False)
df_prop.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.pkl')

# Fetch property_types table
cursor.execute("SELECT * FROM property_types;")
df_prop_type = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df_prop_type.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.csv', index=False)
df_prop_type.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.pkl')

# Close connection
cursor.close()
connection.close()

In [ ]:
#Import property table from SalesForce

import pandas as pd
from pathlib import Path
import os

def process_property_sizes(input_file: Path) -> pd.DataFrame:
    # Define fields to cast as int
    int_fields = [
        'account_id',
        'managed_account_id',
        'Parking Space Count'
        # Add any other fields that should be int
    ]

    # Load CSV with explicit encoding
    df_prop_size = pd.read_csv(input_file, encoding='latin-1')

    # Cast specified fields to int, handling any errors
    for field in int_fields:
        if field in df_prop_size.columns:
            df_prop_size[field] = pd.to_numeric(df_prop_size[field], errors='coerce').fillna(0).astype(int)

    # Save pickle to same directory as input file
    prop_size_pickle_path = input_file.parent / 'df_prop_size.pkl'
    df_prop_size.to_pickle(prop_size_pickle_path)

    # Print info about the conversions
    print("\nData types after conversion:")
    print(df_prop_size[int_fields].dtypes)

    return df_prop_size

if __name__ == "__main__":
    # Define base directory
    data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')

    # Input path
    prop_size_input = data_dir / 'Properties Table jan2025.csv'

    # Run workflow
    df_prop_size = process_property_sizes(prop_size_input)

## Clean data

In [ ]:
# Here are the dfs Ill be working with

df_logs = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs.pkl') # Event data from Splunk
df_prop = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.pkl') # Property metadata from AWS
df_prop_size = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_size.pkl') # Property size data from SalesForce
df_prop_type = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.pkl') # Property metadata from AWS

###This is what the JSON field looks like



{"connectorId":1,"transactionId":1417592169,"meterValue":[{"timestamp":"2025-01-14T13:27:37.145Z","sampledValue":[{"value":"31323855.0","context":"Sample.Periodic","format":"Raw","measurand":"Energy.Active.Import.Register","phase":"L1","location":"Outlet","unit":"Wh"},{"value":"0.00","context":"Sample.Periodic","format":"Raw","measurand":"Current.Import","phase":"L1","location":"Outlet","unit":"A"},{"value":"240.57","context":"Sample.Periodic","format":"Raw","measurand":"Voltage","phase":"L1","location":"Outlet","unit":"V"},{"value":"28","context":"Sample.Periodic","format":"Raw","measurand":"Temperature","phase":null,"location":"Body","unit":"Celsius"},{"value":"6.00","context":"Sample.Periodic","format":"Raw","measurand":"Current.Offered","phase":"L1","location":"Outlet","unit":"A"},{"value":"1440.00","context":"Sample.Periodic","format":"Raw","measurand":"Power.Offered","phase":"L1","location":"Outlet","unit":"W"},{"value":"0.00","context":"Sample.Periodic","format":"Raw","measurand":"Power.Active.Import","phase":"L1","location":"Outlet","unit":"W"}]}]}

In [ ]:
# Unpack the JSON field in the log file
import pandas as pd
import json

def expand_message_json(df):
    rows = []

    for idx, row in df.iterrows():
        # Parse the JSON message
        message = json.loads(row['cleaned_message']) if pd.notna(row['cleaned_message']) else {}

        # Get transactionId from the message
        transaction_id = message.get('transactionId')

        # Extract meter values
        meter_values = message.get('meterValue', [])
        for meter in meter_values:
            timestamp = meter.get('timestamp')
            sampled_values = meter.get('sampledValue', [])

            # Filter for only A and W units
            for sample in sampled_values:
                unit = sample.get('unit')
                if unit in ['A', 'W']:
                    rows.append({
                        'property_id': row['property_id'],
                        'user_id': row['user_id'],
                        'transaction_id': transaction_id,  # Fixed variable name here
                        'timestamp': timestamp,
                        'value': sample.get('value'),
                        'unit': unit
                    })

    return pd.DataFrame(rows)

# Expand the JSON column
df_logs_exp = expand_message_json(df_logs)

# Save the expanded data
df_logs_exp.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')

# Expand the JSON column
df_logs_exp = expand_message_json(df_logs)

# Save the expanded data
df_logs_exp.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')


In [ ]:
# Create a sample of log data

import pandas as pd
from pathlib import Path

# Load the DataFrame
data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')
logs_pickle = data_dir / 'df_logs_exp.pkl'

# Read pickle and sample 10 records
df_logs_exp = pd.read_pickle(logs_pickle)
sample_df = df_logs_exp.sample(n=10, random_state=42)  # random_state for reproducibility

# Save sample to CSV in same directory
sample_df.to_csv(data_dir / 'logs_sample.csv', index=False)

print("10 record sample saved to logs_sample.csv")

In [ ]:
# Fix field name and datatype inconsistencies
import pandas as pd
from pathlib import Path

def normalize_timestamp(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Convert to datetime and truncate to seconds
   df['_time'] = pd.to_datetime(df['_time']).dt.floor('s')

   # Save back to pickle
   df.to_pickle(df_path)

   print("Timestamps normalized and saved")

def normalize_account_ids(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Cast ID fields to int
   df['managed_account_id'] = pd.to_numeric(df['managed_account_id'], errors='coerce').fillna(0).astype(int)
   df['account_id'] = pd.to_numeric(df['account_id'], errors='coerce').fillna(0).astype(int)

   # Save back to pickle
   df.to_pickle(df_path)

   print("Account IDs normalized and saved")

def normalize_prop_size_id(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Cast ID field to int
   df['managed_account_id'] = pd.to_numeric(df['managed_account_id'], errors='coerce').fillna(0).astype(int)

   # Save back to pickle
   df.to_pickle(df_path)

   print("Property size ID normalized and saved")

if __name__ == "__main__":
   # Define base directory
   data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')

   # Define pickle files
   logs_pickle = data_dir / 'df_logs.pkl'
   prop_pickle = data_dir / 'df_prop.pkl'
   prop_size_pickle = data_dir / 'df_prop_size.pkl'

   # Run normalizations
   normalize_timestamp(logs_pickle)
   normalize_account_ids(prop_pickle)
   normalize_prop_size_id(prop_size_pickle)

In [ ]:
# Check join logic
# Does df_logs.property_ud join with df_prop.id ?
# Sample value 3436570000094511023
# Found 20 property_ids in logs that don't exist in properties table

import pandas as pd
from pathlib import Path

# Load the DataFrames
data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')
logs_pickle = data_dir / 'df_logs.pkl'
prop_pickle = data_dir / 'df_prop.pkl'

df_logs = pd.read_pickle(logs_pickle)
df_prop = pd.read_pickle(prop_pickle)

# Check unique values in each column
n_log_props = df_logs['property_id'].nunique()
n_prop_ids = df_prop['id'].nunique()

print(f"Unique property_ids in logs: {n_log_props}")
print(f"Unique ids in properties: {n_prop_ids}")

# Check which property_ids in logs don't exist in properties
missing_props = set(df_logs['property_id'].unique()) - set(df_prop['id'].unique())

if missing_props:
    print(f"\nFound {len(missing_props)} property_ids in logs that don't exist in properties table")
    print("Sample of missing ids:", list(missing_props)[:5])
else:
    print("\nAll property_ids in logs exist in properties table")

# Check actual join
merged_df = df_logs.merge(df_prop, left_on='property_id', right_on='id', how='left')
n_unmatched = merged_df['id'].isna().sum()

print(f"\nUnmatched rows after join: {n_unmatched} ({(n_unmatched/len(df_logs))*100:.2f}% of logs)")


In [ ]:
# Sample missing data

import psycopg2
import pandas as pd
import os

# Load credentials
def load_credentials(path_to_credentials):
   with open(path_to_credentials, 'r') as file:
       for line in file:
           if '=' in line:
               key, value = line.split('=', 1)
               os.environ[key.strip()] = value.strip()

# Connection parameters
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

connection_params = {
   'host': os.getenv('DB_HOST'),
   'dbname': os.getenv('DB_NAME'),
   'user': os.getenv('DB_USER'),
   'password': os.getenv('DB_PASSWORD'),
   'port': os.getenv('DB_PORT')
}

# Connect and fetch data
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Fetch full rows where property_type is NULL
query = "SELECT * FROM properties WHERE property_type IS NULL;"

cursor.execute(query)
columns = [desc[0] for desc in cursor.description]
missing_prop_types = pd.DataFrame(cursor.fetchall(), columns=columns)

print(missing_prop_types.to_string())

# Close connection
cursor.close()
connection.close()

In [ ]:
# Check join logic
# Does df_logs.property_id join with df_prop_size.id

# Check join keys
print("Unique property_id in df_logs:", df_logs['property_id'].nunique())
print("Unique ids in df_prop_size:", df_prop_size['id'].nunique())

# Check overlap between keys
common_keys = set(df_logs['property_id']) & set(df_prop_size['id'])
print("\nNumber of common property IDs:", len(common_keys))

# Percentage of matching keys
logs_match_percentage = len(common_keys) / df_logs['property_id'].nunique() * 100
prop_size_match_percentage = len(common_keys) / df_prop_size['id'].nunique() * 100

print(f"\nPercentage of property_id in logs that match prop_size: {logs_match_percentage:.2f}%")
print(f"Percentage of ids in prop_size that match logs: {prop_size_match_percentage:.2f}%")

# Sample of matching and non-matching keys
print("\nSample of matching property IDs (first 10):")
print(list(common_keys)[:10])

# Check for any non-matching keys
non_matching_in_logs = set(df_logs['property_id']) - set(df_prop_size['id'])
non_matching_in_prop_size = set(df_prop_size['id']) - set(df_logs['property_id'])

print("\nNumber of property_ids in logs not in prop_size:", len(non_matching_in_logs))
print("\nSample of non-matching property_ids in logs (first 10):")
print(list(non_matching_in_logs)[:10])

# Perform a left join to see unmatched rows
merged_df = df_logs.merge(df_prop_size, left_on='property_id', right_on='id', how='left', indicator=True)
print("\nMerge result:")
print(merged_df['_merge'].value_counts())

In [ ]:
# Check join logic
# Check join between df_prop.property_type and df_prop_type.id

print("Unique property_type in df_prop:", df_prop['property_type'].nunique())
print("Unique ids in df_prop_type:", df_prop_type['id'].nunique())

# Check overlap between keys
common_keys = set(df_prop['property_type']) & set(df_prop_type['id'])
print("\nNumber of common property types:", len(common_keys))

# Percentage of matching keys
prop_match_percentage = len(common_keys) / df_prop['property_type'].nunique() * 100
prop_type_match_percentage = len(common_keys) / df_prop_type['id'].nunique() * 100

print(f"\nPercentage of property_type in prop that match prop_type: {prop_match_percentage:.2f}%")
print(f"Percentage of ids in prop_type that match prop: {prop_type_match_percentage:.2f}%")

# Perform a left join to see unmatched rows
merged_df = df_prop.merge(df_prop_type, left_on='property_type', right_on='id', how='left', indicator=True)
print("\nMerge result:")
print(merged_df['_merge'].value_counts())

# Check unmatched property types
unmatched = merged_df[merged_df['_merge'] == 'left_only']
print("\nUnmatched property types:")
print(unmatched['property_type'].unique())

In [ ]:
# Decorate log data with property metadata

df_logs_exp = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')

df_logs_metadata = df_logs_exp.copy()

# Left join to append property_type from df_prop
df_logs_metadata = df_logs_metadata.merge(
   df_prop[['id', 'property_type']],
   left_on='property_id',
   right_on='id',
   how='left'
)

# Drop the redundant id column
df_logs_metadata = df_logs_metadata.drop(columns=['id'])

# Left join to append name from df_prop_type
df_logs_metadata = df_logs_metadata.merge(
   df_prop_type[['id', 'name']],
   left_on='property_type',
   right_on='id',
   how='left'
)

# Drop the redundant id column
df_logs_metadata = df_logs_metadata.drop(columns=['id'])

# Left join to append Parking Space Count from df_prop_size
df_logs_metadata = df_logs_metadata.merge(
   df_prop_size[['id', 'Parking Space Count']],
   left_on='property_id',
   right_on='id',
   how='left'
)

# Save the data
df_logs_metadata.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_metadata.pkl')


In [ ]:
# Data clean-up

import pandas as pd
import pytz

def is_valid_timestamp(timestamp):
    try:
        pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%fZ')
        return True
    except:
        try:
            pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%SZ')
            return True
        except:
            try:
                pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%f%z')
                return True
            except:
                return False

def remove_invalid_timestamps(df):
    return df[df['timestamp'].apply(is_valid_timestamp)]

def truncate_timestamp(df):
    df['timestamp'] = df['timestamp'].str[:16]
    return df

def convert_to_pacific_time(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_convert('US/Pacific')
    return df

def process_timestamps(df):
    df = remove_invalid_timestamps(df)
    df = truncate_timestamp(df)
    df = convert_to_pacific_time(df)
    return df

def clean_and_process_data(df):
    # Drop unnecessary columns
    df = df.drop(columns=['id', 'property_type'])

    # Reorder and select columns
    df = df[['property_id', 'name', 'Parking Space Count', 'transaction_id', 'timestamp', 'user_id', 'unit', 'value']]

    # Rename columns
    df = df.rename(columns={'name': 'property_type', 'Parking Space Count': 'property_size'})

    # Cast numeric formats
    df['property_size'] = pd.to_numeric(df['property_size'], errors='coerce').fillna(0).astype(int)
    df['value'] = pd.to_numeric(df['value'], errors='coerce').fillna(0).astype(int)

    # Process timestamps
    df = process_timestamps(df)

    return df

# Main execution
def main():
    # Load the data
    df_logs_metadata = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_metadata.pkl')

    # Clean and process the data
    df_logs_metadata_processed = clean_and_process_data(df_logs_metadata)

    # Save the processed data
    df_logs_metadata_processed.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_metadata.pkl')

    # Verify the processed data
    print("Processed DataFrame Info:")
    print(df_logs_metadata_processed.info())
    print("\nFirst few rows of processed data:")
    print(df_logs_metadata_processed.head())

if __name__ == "__main__":
    main()

## Data Exploration

In [ ]:
df_logs = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs.pkl') # Event data from Splunk
df_prop = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.pkl') # Property metadata from AWS
df_prop_size = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_size.pkl') # Property size data from SalesForce
df_prop_type = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.pkl') # Property metadata from AWS
df_logs_metadata = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_metadata.pkl') # Decorated log data


In [ ]:
df_logs_metadata.info()

In [ ]:
# Does each transaction_id have a unique timestamp ?

import pandas as pd

# Create a new column with the combination of transaction_id and timestamp
df_logs_metadata['transaction_timestamp_pair'] = df_logs_metadata['transaction_id'].astype(str) + '_' + df_logs_metadata['timestamp'].astype(str)

# Count the number of occurrences of each pair
pair_counts = df_logs_metadata['transaction_timestamp_pair'].value_counts()

# Check if there are any duplicates
duplicates = pair_counts[pair_counts > 1]

if duplicates.empty:
    print("transaction_id and timestamp pairs are unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate transaction_id and timestamp pairs.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique pairs: {len(pair_counts)}")
print(f"Are all pairs unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_pairs = duplicates.index[:5]  # Get the first 5 duplicate pairs
    for pair in duplicate_pairs:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'] == pair][['transaction_id', 'timestamp', 'property_id', 'user_id', 'value', 'unit']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'].isin(duplicates.index)]

    print("\nAnalysis of duplicate pairs:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")
    print(f"Number of unique units in duplicates: {duplicate_df['unit'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())

In [ ]:
# Is the combination of transaction_id, timestamp and unit unique ?
import pandas as pd

# Assuming df_logs_metadata is your existing DataFrame

# Create a new column with the combination of transaction_id, timestamp, and unit
df_logs_metadata['transaction_timestamp_unit_combo'] = (
    df_logs_metadata['transaction_id'].astype(str) + '_' +
    df_logs_metadata['timestamp'].astype(str) + '_' +
    df_logs_metadata['unit']
)

# Count the number of occurrences of each combination
combo_counts = df_logs_metadata['transaction_timestamp_unit_combo'].value_counts()

# Check if there are any duplicates
duplicates = combo_counts[combo_counts > 1]

if duplicates.empty:
    print("The combination of transaction_id, timestamp, and unit is unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate combinations of transaction_id, timestamp, and unit.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique combinations: {len(combo_counts)}")
print(f"Are all combinations unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_combos = duplicates.index[:5]  # Get the first 5 duplicate combinations
    for combo in duplicate_combos:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_unit_combo'] == combo][['transaction_id', 'timestamp', 'unit', 'property_id', 'user_id', 'value']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_unit_combo'].isin(duplicates.index)]

    print("\nAnalysis of duplicate combinations:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())




# Example of a "duplicate" transaction_id: 1938384688
result = df_logs_metadata[df_logs_metadata['transaction_id'] == 1938384688]
print(result)






In [ ]:
# Example of a "duplicate" transaction_id: 1938384688
result = df_logs_metadata[df_logs_metadata['transaction_id'] == 1938384688]
print(result)



In [ ]:
# How often does transaction_id have multiple user_ids ?
# Only 1 example was found.

# Count unique user_ids for each transaction_id
user_id_counts = df_logs_metadata.groupby('transaction_id')['user_id'].nunique()

# Count unique property_ids for each transaction_id
property_id_counts = df_logs_metadata.groupby('transaction_id')['property_id'].nunique()

# Calculate results
total_transactions = len(df_logs_metadata['transaction_id'].unique())
transactions_with_multiple_users = (user_id_counts > 1).sum()
transactions_with_multiple_properties = (property_id_counts > 1).sum()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions with multiple user_ids: {transactions_with_multiple_users}")
print(f"Percentage of transactions with multiple user_ids: {transactions_with_multiple_users/total_transactions*100:.2f}%")
print(f"Transactions with multiple property_ids: {transactions_with_multiple_properties}")
print(f"Percentage of transactions with multiple property_ids: {transactions_with_multiple_properties/total_transactions*100:.2f}%")

# If you want to see examples of transactions with multiple user_ids or property_ids:
print("\nExamples of transactions with multiple user_ids:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(user_id_counts[user_id_counts > 1].index)].groupby('transaction_id')[['user_id', 'property_id']].head())

print("\nExamples of transactions with multiple property_ids:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(property_id_counts[property_id_counts > 1].index)].groupby('transaction_id')[['user_id', 'property_id']].head())

In [ ]:
# How often does transaction_id have multiple year or month values ?
# Looks like its concentrated at the begining or ending of a month

# Extract year and month from the timestamp
df_logs_metadata['year'] = df_logs_metadata['timestamp'].dt.year
df_logs_metadata['month'] = df_logs_metadata['timestamp'].dt.month

# Count unique years for each transaction_id
year_counts = df_logs_metadata.groupby('transaction_id')['year'].nunique()

# Count unique months for each transaction_id
month_counts = df_logs_metadata.groupby('transaction_id')['month'].nunique()

# Calculate results
total_transactions = len(df_logs_metadata['transaction_id'].unique())
transactions_with_multiple_years = (year_counts > 1).sum()
transactions_with_multiple_months = (month_counts > 1).sum()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions spanning multiple years: {transactions_with_multiple_years}")
print(f"Percentage of transactions spanning multiple years: {transactions_with_multiple_years/total_transactions*100:.2f}%")
print(f"Transactions spanning multiple months: {transactions_with_multiple_months}")
print(f"Percentage of transactions spanning multiple months: {transactions_with_multiple_months/total_transactions*100:.2f}%")

# If you want to see examples of transactions spanning multiple years or months:
print("\nExamples of transactions spanning multiple years:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(year_counts[year_counts > 1].index)].groupby('transaction_id')[['timestamp', 'year', 'month']].head())

print("\nExamples of transactions spanning multiple months:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(month_counts[month_counts > 1].index)].groupby('transaction_id')[['timestamp', 'year', 'month']].head())

In [ ]:
# How often does transaction_id have multiple A or W non-0 values  ?

# Filter for non-zero 'A' and 'W' values
df_non_zero = df_logs_metadata[(df_logs_metadata['value'] != 0) &
                               (df_logs_metadata['unit'].isin(['A', 'W']))]

# Group by transaction_id and unit, then count non-zero values
value_counts = df_non_zero.groupby(['transaction_id', 'unit'])['value'].count().unstack(fill_value=0)

# Count transactions with multiple non-zero 'A' values
multiple_A = (value_counts['A'] > 1).sum()

# Count transactions with multiple non-zero 'W' values
multiple_W = (value_counts['W'] > 1).sum()

# Total unique transactions
total_transactions = df_logs_metadata['transaction_id'].nunique()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions with multiple non-zero 'A' values: {multiple_A}")
print(f"Percentage of transactions with multiple non-zero 'A' values: {multiple_A/total_transactions*100:.2f}%")
print(f"Transactions with multiple non-zero 'W' values: {multiple_W}")
print(f"Percentage of transactions with multiple non-zero 'W' values: {multiple_W/total_transactions*100:.2f}%")

# Examples of transactions with multiple non-zero 'A' or 'W' values
print("\nExamples of transactions with multiple non-zero 'A' values:")
print(df_non_zero[df_non_zero['transaction_id'].isin(value_counts[value_counts['A'] > 1].index) &
                  (df_non_zero['unit'] == 'A')].groupby('transaction_id').head())

print("\nExamples of transactions with multiple non-zero 'W' values:")
print(df_non_zero[df_non_zero['transaction_id'].isin(value_counts[value_counts['W'] > 1].index) &
                  (df_non_zero['unit'] == 'W')].groupby('transaction_id').head())




In [ ]:
# Check uniqueness of transaction_id across different column combinations
def check_uniqueness(df, columns):
    unique_combinations = df.groupby(columns)['transaction_id'].nunique()
    print(f"\nUniqueness check for {columns}:")
    print("Number of unique transaction_ids per group:", unique_combinations.max())

    if unique_combinations.max() > 1:
        print("\nGroups with multiple transaction_ids:")
        print(unique_combinations[unique_combinations > 1])

# Check various column combinations
check_columns = [
    ['user_id', 'timestamp', 'property_id', 'value', 'unit'],
    ['user_id', 'property_id', 'value', 'unit'],
    ['timestamp', 'property_id', 'value', 'unit'],
    ['user_id', 'timestamp', 'value', 'unit']
]

for cols in check_columns:
    check_uniqueness(df_logs_metadata, cols)

# Additional overall statistics
print("\nOverall unique counts:")
print("Total rows:", len(df_logs_metadata))
print("Unique transaction_ids:", df_logs_metadata['transaction_id'].nunique())
print("Unique combinations:",
    df_logs_metadata.groupby(['user_id', 'timestamp', 'property_id', 'value', 'unit'])['transaction_id'].nunique().max())

In [ ]:
import pandas as pd

# Assuming df_logs_metadata is your existing DataFrame

# Create a new column with the combination of transaction_id and timestamp
df_logs_metadata['transaction_timestamp_pair'] = df_logs_metadata['transaction_id'].astype(str) + '_' + df_logs_metadata['timestamp'].astype(str)

# Count the number of occurrences of each pair
pair_counts = df_logs_metadata['transaction_timestamp_pair'].value_counts()

# Check if there are any duplicates
duplicates = pair_counts[pair_counts > 1]

if duplicates.empty:
    print("transaction_id and timestamp pairs are unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate transaction_id and timestamp pairs.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique pairs: {len(pair_counts)}")
print(f"Are all pairs unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_pairs = duplicates.index[:5]  # Get the first 5 duplicate pairs
    for pair in duplicate_pairs:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'] == pair][['transaction_id', 'timestamp', 'property_id', 'user_id', 'value', 'unit']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'].isin(duplicates.index)]

    print("\nAnalysis of duplicate pairs:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")
    print(f"Number of unique units in duplicates: {duplicate_df['unit'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())import matplotlib.pyplot as plt
import numpy as np

# Histogram of timestamps per transaction_id
timestamps_per_transaction = df_logs_enriched.groupby('transaction_id')['timestamp'].nunique()

plt.figure(figsize=(12, 5))

# Left plot - log scale
plt.subplot(1, 2, 1)
plt.hist(timestamps_per_transaction, bins=np.logspace(np.log10(1), np.log10(timestamps_per_transaction.max()), 50))
plt.xscale('log')
plt.yscale('log')
plt.title('Unique Timestamps per Transaction ID (Log Scale)')
plt.xlabel('Number of Unique Timestamps')
plt.ylabel('Count of Transaction IDs')
plt.grid(True)

# Right plot - properties per transaction
properties_per_transaction = df_logs_enriched.groupby('transaction_id')['property_id'].nunique()
plt.subplot(1, 2, 2)
plt.hist(properties_per_transaction, bins=50)
plt.title('Unique Properties per Transaction ID')
plt.xlabel('Number of Unique Properties')
plt.ylabel('Count of Transaction IDs')

plt.tight_layout()
plt.show()

print("\nTimestamps per transaction summary:")
print(timestamps_per_transaction.describe())
print("\nProperties per transaction summary:")
print(properties_per_transaction.describe())

# Print additional context
print("\nTotal number of unique transactions:", len(timestamps_per_transaction))
print("Total number of unique properties:", df_logs_enriched['property_id'].nunique())
print("Total number of timestamps:", df_logs_enriched['timestamp'].nunique())

## Engineer Features

In [ ]:
import pandas as pd
import pytz
import numpy as np
from datetime import datetime

def convert_to_pst_components(df):
    """Convert timestamp to PST and extract components."""
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601')
    df['timestamp'] = df['timestamp'].dt.tz_convert('US/Pacific')

    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute

    return df

def add_day_info(df):
    """Add day of week and weekend indicator."""
    df['day_of_week'] = df['timestamp'].dt.dayofweek + 1
    df['day_weekend'] = (df['day_of_week'] >= 6).astype(int)
    return df

def get_nearest_holiday(df):
    """Calculate proximity to major US holidays."""
    major_holidays = {
        '2024-01-01': "New Year's Day",
        '2024-01-15': "Martin Luther King Jr. Day",
        '2024-02-19': "Presidents Day",
        '2024-05-27': "Memorial Day",
        '2024-07-04': "Independence Day",
        '2024-09-02': "Labor Day",
        '2024-11-28': "Thanksgiving",
        '2024-12-25': "Christmas",
        '2025-01-01': "New Year's Day",
        '2025-01-20': "Martin Luther King Jr. Day",
        '2025-02-17': "Presidents Day",
        '2025-05-26': "Memorial Day",
        '2025-07-04': "Independence Day",
        '2025-09-01': "Labor Day",
        '2025-11-27': "Thanksgiving",
        '2025-12-25': "Christmas"
    }

    holiday_dates = pd.to_datetime(list(major_holidays.keys())).sort_values()
    holiday_dates_array = holiday_dates.values
    dates_array = pd.to_datetime(df['timestamp'].dt.date.unique()).values
    holiday_lookup = {}

    for date in dates_array:
        days_diff = np.abs((holiday_dates_array - date).astype('timedelta64[D]').astype(int))
        closest_idx = np.argmin(days_diff)
        closest_date = holiday_dates[closest_idx]

        holiday_lookup[pd.Timestamp(date).date()] = {
            'days_to_nearest_holiday': days_diff[closest_idx],
            'nearest_holiday_date': closest_date,
            'nearest_holiday_name': major_holidays[closest_date.strftime('%Y-%m-%d')]
        }

    # Create and merge holiday information
    df['date'] = df['timestamp'].dt.date
    result = pd.DataFrame.from_dict(holiday_lookup, orient='index')
    result.index = pd.to_datetime(result.index).date
    df = df.merge(result, left_on='date', right_index=True)
    df = df.drop('date', axis=1)

    return df

def process_timestamps(input_path, output_path):
    """Main function to process all timestamp-related features."""
    df = pd.read_pickle(input_path)
    df = convert_to_pst_components(df)
    df = add_day_info(df)
    df = get_nearest_holiday(df)
    df.to_pickle(output_path)
    return df

if __name__ == "__main__":
    input_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched.pkl'
    output_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_holidays.pkl'

    df_processed = process_timestamps(input_path, output_path)
    print(f"Processing complete. File saved to: {output_path}")

In [ ]:

# Add user and usage data
import pandas as pd

def add_unique_user_counts(df, group_cols, user_col):
    """Count unique users per group."""
    user_counts = df.groupby(group_cols)[user_col].nunique().reset_index()
    user_counts.rename(columns={user_col: 'unique_user_count'}, inplace=True)
    df = df.merge(user_counts, on=group_cols, how='left')
    return df

def add_usage_sums(df, group_cols):
    """Add sums of values for each unit type by group."""
    # Calculate sums for each unit type
    sums = df.groupby([*group_cols, 'unit_encoded'])['value'].sum().reset_index()

    # Pivot to create separate columns for A and W
    sums = sums.pivot(
        index=group_cols,
        columns='unit_encoded',
        values='value'
    ).reset_index()

    # Rename columns
    sums.rename(
        columns={
            0: 'sum_value_A',   # Amps were encoded as 0
            1: 'sum_value_Wh'   # Watts were encoded as 1
        },
        inplace=True
    )

    return df.merge(sums, on=group_cols, how='left')

if __name__ == "__main__":
    input_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_holidays.pkl'
    output_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl'

    # Read data
    df = pd.read_pickle(input_path)

    # Extract datetime components
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour

    # Define grouping columns after datetime components are created
    group_cols = ['property_id', 'year', 'month', 'day', 'hour']

    # Add user and usage metrics
    df = add_unique_user_counts(df, group_cols, 'user_id')
    df = add_usage_sums(df, group_cols)

    # Save results
    df.to_pickle(output_path)
    print(f"Processing complete. File saved to: {output_path}")

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

In [ ]:
df.info()

In [ ]:
# Halt
sys.exit()

# Prep df for analysis

## Check for colinearity

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert columns to numeric, handling errors
columns_to_analyze = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Create a clean dataframe for analysis
df_clean = df[columns_to_analyze].copy()

# Convert each column to numeric, handling errors
for col in columns_to_analyze:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # Print info about conversion
    print(f"\nColumn: {col}")
    print(f"Null values after conversion: {df_clean[col].isnull().sum()}")
    print(f"Sample unique values: {df_clean[col].dropna().sample(5).tolist()}")

# Calculate correlations for cleaned numeric columns
correlations = df_clean.corr()

# Create correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

# Basic summary statistics
print("\nSummary Statistics:")
print(df_clean.describe())

# Check for patterns across categorical variables
print("\nMean energy consumption (sum_value_Wh) by:")
print("\nDay of Week:")
print(df_clean.groupby('day_of_week')['sum_value_Wh'].mean().sort_values(ascending=False))

print("\nHour of Day:")
print(df_clean.groupby('hour')['sum_value_Wh'].mean().sort_values(ascending=False))

print("\nMonth:")
print(df_clean.groupby('month')['sum_value_Wh'].mean().sort_values(ascending=False))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load and clean data
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert columns to numeric
df['hour'] = pd.to_numeric(df['hour'], errors='coerce')
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')
df['unique_user_count'] = pd.to_numeric(df['unique_user_count'], errors='coerce')

# Group by hour and calculate various statistics
hourly_stats = df.groupby('hour').agg({
    'sum_value_Wh': ['mean', 'median', 'std', 'count'],
    'unique_user_count': 'mean'
}).round(2)

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Energy consumption pattern
ax1.plot(hourly_stats[('sum_value_Wh', 'mean')], marker='o')
ax1.fill_between(range(24),
                 hourly_stats[('sum_value_Wh', 'mean')] - hourly_stats[('sum_value_Wh', 'std')],
                 hourly_stats[('sum_value_Wh', 'mean')] + hourly_stats[('sum_value_Wh', 'std')],
                 alpha=0.2)
ax1.set_title('24-Hour Energy Consumption Pattern')
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Average Energy Consumption (Wh)')
ax1.grid(True)

# Plot 2: Users vs Energy
ax2.scatter(hourly_stats[('unique_user_count', 'mean')],
           hourly_stats[('sum_value_Wh', 'mean')],
           alpha=0.6)
# Add hour labels to each point
for i in range(24):
    ax2.annotate(str(i),
                (hourly_stats[('unique_user_count', 'mean')][i],
                 hourly_stats[('sum_value_Wh', 'mean')][i]))
ax2.set_title('Users vs Energy Consumption by Hour')
ax2.set_xlabel('Average Number of Users')
ax2.set_ylabel('Average Energy Consumption (Wh)')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Print detailed statistics for peak hours
peak_hours = hourly_stats.nlargest(5, ('sum_value_Wh', 'mean'))
print("\nPeak Hours Analysis:")
print(peak_hours)

# Calculate relative increase during peak vs off-peak
off_peak_mean = hourly_stats.loc[0:6, ('sum_value_Wh', 'mean')].mean()
peak_mean = hourly_stats.loc[16:19, ('sum_value_Wh', 'mean')].mean()
increase_factor = peak_mean / off_peak_mean

print(f"\nPeak vs Off-peak Analysis:")
print(f"Average off-peak consumption (midnight-6am): {off_peak_mean:.2f} Wh")
print(f"Average peak consumption (4pm-7pm): {peak_mean:.2f} Wh")
print(f"Peak hours consume {increase_factor:.1f}x more energy than off-peak hours")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert sum_value_Wh to numeric
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')

# Create a single property type column for easier plotting
prop_type_cols = [col for col in df.columns if col.startswith('prop_type_')]
df['property_type'] = np.nan

for col in prop_type_cols:
    mask = df[col] == 1
    df.loc[mask, 'property_type'] = col.replace('prop_type_', '')

# Create box plot
plt.figure(figsize=(15, 8))
sns.boxplot(x='property_type', y='sum_value_Wh', data=df)
plt.xticks(rotation=45, ha='right')
plt.title('Energy Consumption Distribution by Property Type')
plt.xlabel('Property Type')
plt.ylabel('Energy Consumption (Wh)')
plt.tight_layout()
plt.show()

# Print basic stats for each property type
print("\nBasic statistics by property type:")
stats = df.groupby('property_type')['sum_value_Wh'].describe()
print(stats)

# Count number of observations for each property type
print("\nNumber of observations per property type:")
counts = df['property_type'].value_counts()
print(counts)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert to numeric
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')
df['unique_user_count'] = pd.to_numeric(df['unique_user_count'], errors='coerce')

# Create a single property type column
prop_type_cols = [col for col in df.columns if col.startswith('prop_type_')]
df['property_type'] = np.nan

for col in prop_type_cols:
    mask = df[col] == 1
    df.loc[mask, 'property_type'] = col.replace('prop_type_', '')

# Calculate energy per user
df['energy_per_user'] = df['sum_value_Wh'] / df['unique_user_count']

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Plot 1: Total Energy
sns.boxplot(x='property_type', y='sum_value_Wh', data=df, ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.set_title('Total Energy Consumption by Property Type')
ax1.set_xlabel('Property Type')
ax1.set_ylabel('Energy Consumption (Wh)')

# Plot 2: Energy per User
sns.boxplot(x='property_type', y='energy_per_user', data=df, ax=ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.set_title('Energy Consumption per User by Property Type')
ax2.set_xlabel('Property Type')
ax2.set_ylabel('Energy Consumption per User (Wh/user)')

plt.tight_layout()
plt.show()

# Print statistics
print("\nStatistics by property type:")
stats = df.groupby('property_type').agg({
    'sum_value_Wh': ['count', 'mean'],
    'unique_user_count': 'mean',
    'energy_per_user': 'mean'
}).round(2)

print(stats)

In [ ]:
# First, let's look at the raw data before any processing
print("Original data counts by property type:")
for col in prop_type_cols:
    print(f"\n{col}:")
    print(f"Number of 1s: {df[col].sum()}")
    print(f"Number of non-null values: {df[col].count()}")

# Let's also check for nulls in key columns
print("\nNull values in key columns:")
print(df[['sum_value_Wh', 'unique_user_count']].isnull().sum())

# Let's look at the data before any type conversion
print("\nSample of raw data before conversion:")
sample_data = df[['property_type', 'sum_value_Wh', 'unique_user_count']].head(10)
print(sample_data)

# Check data types of key columns
print("\nData types of columns:")
print(df.dtypes)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load your dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

def clean_numeric_string(value):
    """Clean numeric string by taking the first valid number."""
    if pd.isna(value):
        return 0
    # Convert to string if not already
    value = str(value)
    # Find first number (integer or decimal)
    parts = value.split('.')
    if not parts:
        return 0
    try:
        # Take first valid number
        return float(parts[0])
    except ValueError:
        return 0

# Clean and convert sum_value columns
df['sum_value_A'] = df['sum_value_A'].apply(clean_numeric_string).astype(int)
df['sum_value_Wh'] = df['sum_value_Wh'].apply(clean_numeric_string).astype(int)
df['unique_user_count'] = df['unique_user_count'].astype(int)

# Select only numerical features for VIF calculation
numerical_columns = [
    "day_of_week",
    "day_weekend",
    "days_to_nearest_holiday",
    "year",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Prepare the DataFrame for VIF calculation
X = df[numerical_columns].copy()

# Check for NaN and inf values
print(f"NaN values before VIF calculation: {X.isna().sum().sum()}")
print(f"Inf values before VIF calculation: {((X == np.inf) | (X == -np.inf)).sum().sum()}")

# Handle NaN and inf values
X = X.fillna(0)  # Replace NaN values with 0 or other strategy
X.replace([np.inf, -np.inf], 0, inplace=True)

# Check again after handling NaN and inf values
print(f"NaN values after VIF calculation: {X.isna().sum().sum()}")
print(f"Inf values after VIF calculation: {((X == np.inf) | (X == -np.inf)).sum().sum()}")

# Add a constant column for intercept
X['intercept'] = 1

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Drop the constant column after VIF calculation
vif_data = vif_data[vif_data["Feature"] != "intercept"]

# Display the VIF values
print(vif_data)

In [ ]:
# Drop suspect features and run VIF again

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Drop suspect features
df = df.drop(['day_weekend', 'year'], axis=1)

# Define numerical columns
numerical_columns = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Convert columns to numeric type and handle any non-numeric values
X = df[numerical_columns].copy()
for column in numerical_columns:
    # Convert to numeric, coerce any non-numeric values to NaN
    X[column] = pd.to_numeric(X[column], errors='coerce')

    # Fill NaN values with the median of the column
    X[column] = X[column].fillna(X[column].median())

# Add a constant column for intercept
X['intercept'] = 1

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Drop the constant column after VIF calculation
vif_data = vif_data[vif_data["Feature"] != "intercept"]

# Sort VIF values in descending order
vif_data = vif_data.sort_values('VIF', ascending=False)

# Display the VIF values
print("\nVariance Inflation Factors:")
print(vif_data)

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Define the dependent variable (y) - in this case let's use sum_value_Wh as our target
y = "sum_value_Wh"

# Define independent variables (features to compare groups)
independent_vars = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count"
    #"sum_value_A"
]

# Convert columns to numeric type and handle any non-numeric values
X = df[independent_vars].copy()
for column in independent_vars:
    X[column] = pd.to_numeric(X[column], errors='coerce')
    X[column] = X[column].fillna(X[column].median())

# Convert y to numeric
df[y] = pd.to_numeric(df[y], errors='coerce')
df[y] = df[y].fillna(df[y].median())

# Function to run one-way ANOVA for each independent variable against y
def run_anova_with_target(df, independent_vars, y):
    results = []

    for var in independent_vars:
        # Create groups based on the independent variable
        groups = []
        # Create 5 groups using quantiles for continuous variables
        df['group'] = pd.qcut(df[var], q=5, labels=['G1', 'G2', 'G3', 'G4', 'G5'])

        # Get the y values for each group
        for group in df['group'].unique():
            groups.append(df[df['group'] == group][y].values)

        # Perform one-way ANOVA
        f_stat, p_val = stats.f_oneway(*groups)

        results.append({
            'Independent Variable': var,
            'F-statistic': f_stat,
            'p-value': p_val
        })

    return pd.DataFrame(results)

# Run ANOVA
anova_results = run_anova_with_target(df, independent_vars, y)

# Sort results by p-value
anova_results_sorted = anova_results.sort_values('p-value')

# Display results
pd.set_option('display.float_format', lambda x: '{:.10f}'.format(x) if x < 0.0001 else '{:.4f}'.format(x))
print(f"\nOne-way ANOVA Results (dependent variable: {y}):")
print(anova_results_sorted)

# Add significance indicators
anova_results_sorted['Significance'] = ['***' if p < 0.001
                                      else '**' if p < 0.01
                                      else '*' if p < 0.05
                                      else 'ns' for p in anova_results_sorted['p-value']]

print("\nSignificance levels:")
print("***: p < 0.001")
print("**: p < 0.01")
print("*: p < 0.05")
print("ns: not significant")

In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_sampled['unique_user_count'], df_sampled['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_sampled['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_sampled['property_id'] = df_sampled['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_sampled.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_sampled).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)


In [ ]:
# Calculate the IQR for the column with potential outliers
Q1 = df_sampled['hour_sum_value_A'].quantile(0.25)
Q3 = df_sampled['hour_sum_value_A'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df_filtered = df_sampled[(df_sampled['hour_sum_value_A'] >= lower_bound) & (df_sampled['hour_sum_value_A'] <= upper_bound)]


In [ ]:
## Create a property lookup

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table};"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "location",
    "properties"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/{table}_table_extract.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")



In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_filtered['unique_user_count'], df_filtered['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_filtered['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_filtered['property_id'] = df_filtered['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_filtered.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_filtered).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)

In [ ]:
# Decorate data with engineered values

from datetime import datetime
import pytz

# Function to convert to PST and extract datetime
def convert_to_pst_as_datetime(timestamp):
    # Parse the UTC timestamp
    utc_time = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Set timezone to UTC
    utc_time = utc_time.replace(tzinfo=pytz.UTC)
    # Convert to PST
    pst_time = utc_time.astimezone(pytz.timezone('US/Pacific'))
    # Truncate to day, month, year, and hour (zero minutes and seconds)
    return pst_time.replace(minute=0, second=0, microsecond=0)

# Apply the function to convert timestamp
df_a_s_o['time_sample'] = df_a_s_o['timestamp'].apply(convert_to_pst_as_datetime)

# Add a column for day of the week (0 = Monday, 6 = Sunday)
df_a_s_o['day_of_week'] = df_a_s_o['time_sample'].dt.dayofweek

# Add a column for hour of the day (24hr format)
df_a_s_o['hour_of_day'] = df_a_s_o['time_sample'].dt.hour

# Add a column for ISO week number
df_a_s_o['week_number'] = df_a_s_o['time_sample'].dt.isocalendar().week

# Add in count of unique users
df_a_s_o['unique_user_count'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['user_id']
    .transform('nunique')
)

# Add in sum of unit_a
df_a_s_o['sum_of_unit_a'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_a']
    .transform('sum')
)

# Add in sum of watt_h
df_a_s_o['sum_of_unit_wh'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_wh']
    .transform('sum')
)

# Print the updated DataFrame
print(df_a_s_o)

In [ ]:
# Data Check
print(df_a_s_o['week_number'].unique())


# Calculate the overall count of unique user IDs
unique_user_count = df_a_s_o['user_id'].nunique()

# Calculate the sum of unit_a
sum_of_unit_a = df_a_s_o['unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = df_a_s_o['unit_wh'].sum()

# Print the results
print(f"Unique User Count: {unique_user_count}")
print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78

In [ ]:
df_a_s_o.info()

In [ ]:
# Reduce the DataFrame to unique rows based on the specified columns
reduced_df = df_a_s_o.drop_duplicates(
    subset=['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']
)

# Keep only the specified columns
reduced_df = reduced_df[['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']]

# Display the resulting DataFrame
print(reduced_df.info())
print(reduced_df.head())


In [ ]:

# Calculate the sum of unit_a
sum_of_unit_a = reduced_df['sum_of_unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = reduced_df['sum_of_unit_wh'].sum()

# Print the results

print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78



In [ ]:
# Write a local file to take a look

df_a_s_o.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/df_a_s_o.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.regplot(x='unique_user_count', y='sum_of_unit_wh', data=df_a_s_o, scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
plt.xlabel('User unique_user_count Count')
plt.ylabel('Total Unit WH')
plt.title('Regression Plot: User ID Count vs. Total Unit WH')
plt.show()

In [ ]:
df_a_s_o.info()

In [ ]:
# Data is week 32 through week 44 (12)
# So below, there is no week lag1 value for week 32 because it is the first

# Identify the peak total_unit_wh for each week
peak_weekly_data = df.loc[df.groupby('week_number')['sum_of_unit_wh'].idxmax()]

# Sort by week number to ensure correct lagging
peak_weekly_data = peak_weekly_data.sort_values('week_number')

# Add only lag_1 features
peak_weekly_data['lag_1_day_of_week'] = peak_weekly_data['day_of_week'].shift(1)
peak_weekly_data['lag_1_hour'] = peak_weekly_data['hour_of_day'].shift(1)

# Drop rows with insufficient lag (week 1)
peak_weekly_data = peak_weekly_data.dropna()

# Retain only relevant columns
peak_weekly_data = peak_weekly_data[['week_number', 'day_of_week', 'hour_of_day', 'lag_1_day_of_week', 'lag_1_hour']]

print("Updated DataFrame:")
print(peak_weekly_data)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Features (lagged day of week and hour) and target (day of week)
X = peak_weekly_data[['lag_1_day_of_week', 'lag_1_hour']]
y = peak_weekly_data['day_of_week']  # Target: Day of the week

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Day of Week Prediction Accuracy:", accuracy)

# Display true vs predicted values
results = pd.DataFrame({'True Day': y_test, 'Predicted Day': y_pred})
print("\nTrue vs Predicted Days of the Week:")
print(results)


In [ ]:


# Feature importance for day_of_week classification
clf_importances = clf.feature_importances_
plt.barh(X.columns, clf_importances)
plt.title("Feature Importance for Day of Week Prediction")
plt.show()

# Feature importance for hour regression
reg_importances = reg.feature_importances_
plt.barh(X.columns, reg_importances)
plt.title("Feature Importance for Hour Prediction")
plt.show()


## Appendix

####AWS Tables

In [ ]:

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Execute a query to fetch all table names
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """

    cursor.execute(query)
    tables = cursor.fetchall()

    # Print the table names
    for table in tables:
        print(table[0])

except Exception as error:
    print(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        print("Connection closed.")


In [ ]:
# This creates a table of field names and sample values
import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]


# "station_logs" is the big one. WOuld have the same fields/data as MeterValues data in Splunk.




# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Query to fetch the first few rows from the current table
        query = f"SELECT * FROM {table} LIMIT 10;"
        cursor.execute(query)

        # Fetch the rows
        rows = cursor.fetchall()
        # Fetch the column headers
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame from the fetched data
        df = pd.DataFrame(rows, columns=column_names)

        # Prepare the transposed DataFrame
        transposed_data = {
            'Header': column_names,
            'Data Type': [df[col].dtype.name for col in column_names],  # Get the data type
            'Example': [df[col].iloc[0] if not df[col].empty else None for col in column_names]  # Example from the first row
        }

        df_transposed = pd.DataFrame(transposed_data)

        # Write the DataFrame to CSV
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/{table}_fields.csv'
        df_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")

In [ ]:
# This creates a table of sample records

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table} LIMIT 10;"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/{table}_example_data.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")


In [ ]:
# AWS Extract of occp_sessions

import pandas as pd
import glob
import os

def combine_large_csv_files(directory_path, output_filename, chunksize=100):
    # Get all CSV files in the directory
    all_files = glob.glob(os.path.join(directory_path, "*.csv"))

    # Write header from first file
    first_chunk = pd.read_csv(all_files[0], nrows=0)
    first_chunk.to_csv(output_filename, index=False)

    # Process each file
    for i, filename in enumerate(all_files):
        print(f"Processing file {i+1} of {len(all_files)}: {filename}")

        # Process file in chunks
        for chunk in pd.read_csv(filename, chunksize=chunksize):
            # Append chunk to output file without headers
            chunk.to_csv(output_filename,
                        mode='a',
                        header=False,
                        index=False)

    print(f"Successfully combined {len(all_files)} files into {output_filename}")

# Usage
directory = "/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/"  # Replace with your directory path
output_file = "/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/Q4_2024.csv"    # Name for your output file

In [ ]:
# Set Headers:

df_aws.columns = ['qrcode','connector', 'serial_num', 'org_id', 'property_id', 'station_id', 'transaction_id', 'metered_type', 'timestamp', 'metered_value']


# Get the minimum (earliest) datetime
min_datetime = df_aws['timestamp'].min()

# Get the maximum (latest) datetime
max_datetime = df_aws['timestamp'].max()

print("Earliest datetime:", min_datetime)
print("Latest datetime:", max_datetime)

# Now I need to build the correct table directly from RS
Scratch that. Queries are impacting production data. Need to stick with splunk logs....